# Numerical analysis: TP-1 - Arnaud Capitan

In [69]:
import numpy as np

#### 1) Integers and floating point representations
run the following cells below.

In [70]:
a = 2 * np.ones(1).astype(int)
print(f"a is equal to {a} and its type is {a.dtype}.")

a is equal to [2] and its type is int32.


In [71]:
for i in range(60, 68):
    print(f"2 ** {i} = {a ** i}")


print(f"\n2 ** {30} = {a ** 30}")
print(f"2 ** {31} = {a ** 31}")
print(f"2 ** {32} = {a ** 32}")

2 ** 60 = [0]
2 ** 61 = [0]
2 ** 62 = [0]
2 ** 63 = [0]
2 ** 64 = [0]
2 ** 65 = [0]
2 ** 66 = [0]
2 ** 67 = [0]

2 ** 30 = [1073741824]
2 ** 31 = [-2147483648]
2 ** 32 = [0]


**Q1.  Can you explain the observed behavior ?  Propose a way to fix this.**

a is a int32 variable, which means it is stored in a 31 bits variable, with 1 bit for the sign.

If we take a close look at the int class declaration, we see that there is a bit_length attribute to the int class, so we can guess that the size of our int is allocated dynamically according to the calculations done.

class int:
    @overload
    def __new__(cls, x: ConvertibleToInt = ..., /) -> Self: ...
    @overload
    def __new__(cls, x: str | bytes | bytearray, /, base: SupportsIndex) -> Self: ...
    def as_integer_ratio(self) -> tuple[int, Literal[1]]: ...
    @property
    def real(self) -> int: ...
    @property
    def imag(self) -> Literal[0]: ...
    @property
    def numerator(self) -> int: ...
    @property
    def denominator(self) -> Literal[1]: ...
    def conjugate(self) -> int: ...
    def bit_length(self) -> int: ...
    if sys.version_info >= (3, 10):
        def bit_count(self) -> int: ...

In [72]:
a = 2
print(f"a is equal to {a} and its type is {type(a)}.")
for i in range(60, 68):
    print(f"2 ** {i} = {a ** i}")

a is equal to 2 and its type is <class 'int'>.
2 ** 60 = 1152921504606846976
2 ** 61 = 2305843009213693952
2 ** 62 = 4611686018427387904
2 ** 63 = 9223372036854775808
2 ** 64 = 18446744073709551616
2 ** 65 = 36893488147419103232
2 ** 66 = 73786976294838206464
2 ** 67 = 147573952589676412928


**Q2. Does the problem occur without specifying the dtype `np.ones(1)`? Deduce a real numpy usecase where this might be a problem.**

This problem doesn't occur when we don't specify the dtype np.ones(1). It means that the numpy variables inside an array can't exceed 2**31.
It might be an issue when computing in an array the **Mersennes** numbers in an array

$$ M_n = 2^n - 1 $$

 In deep learning applications, chosing the "right" dtype is a very important tradeoff between speed and accuracy.

 ___

#### 2) Imperfect floating point numbers 

Consider the function $f(x) = 2x$ on $[0, 0.5]$ and $f(x) = 2x - 1$ on $]0.5, 1]$ 

**Q1.** Consider the sequence defined by $x_{n+1} = f(x_n)$ with $x_0 = 0.1$ Compute the first 5 elements of the sequence (manually). What do you conclude ?

$$ x_1 = 2*x_0 = 0.2 \text{ because } x_0 \in [0,0.5] $$ 
$$ x_2 = 2*x_1 = 0.4 \text{ because } x_1 \in [0,0.5] $$
$$ x_3 = 2*x_2 = 0.8 \text{ because } x_2 \in [0,0.5] $$
$$ x_4 = 2*x_3 - 1 = 0.6 \text{ because } x_3 \in ]0.5,1] $$
$$ x_5 = 2*x_4 - 1 = 0.2 = x_1 \text{ because } x_4 \in ]0.5,1] $$ 

We can conclude that with $x_0 = 0.1$, the sequence $(x_n)$ defined by $x_{n+1} = f(x_n)$ is periodical, and we have, $\forall n$ in $\textbf{N}$ :

$$ x_{4n+1} = 0.2 $$
$$ x_{4n+2} = 0.4 $$
$$ x_{4n+3} = 0.8 $$
$$ x_{4n+4} = 0.6 $$

**Q2.** Complete the function below that returns $x_n$. What do you observe ?

In [73]:
x0 = 0.1

def f(x, n=100):
    i = 0
    while i < n :
        i+=1
        if x <= 0.5:
            x = 2*x
        else :
            x = 2*x - 1
    return x

print(f(x0, 51))
print(f(x0, 52))
print(f(x0, 53))
print(f(x0, 54))

print(f(x0,100))

0.8125
0.625
0.25
0.5
1.0


By using the computations done above the python function, we have that $f^{100}(x_0) = x_{100} = x_{24*4 + 4} = 0.6$

But using python calculations, the integer approximation error grows so big that the calculation at around n = 50 are going around the fix point 0.5 -> 1 -> 0.5

`float64` numbers are represented using 64 bits as:
$$(-1)^s \quad 0.m_1..m_{52} \quad 2^{e_1..e_{11}}$$
where $s$ is a sign bit, $m$ is the mantissa (52 bits) and $e$ is the exponent (11 bits)

**Q4** Take a moment a contemplate this mystery. Use the `pretty_float_bits` function below to find an explanation for this.

In [74]:
0.1 + 0.2

0.30000000000000004

In [48]:
import struct

def float_to_bin(f) -> str:
    fmt = ">d"
    bz = struct.pack(fmt, f)
    return "".join(f"{b:08b}" for b in bz)

def sign_exponent_fraction(s):
    return s[0:1], s[1:12], s[12:64]

def pretty_float_bits(f) -> str:
    return " ".join(sign_exponent_fraction(float_to_bin(f)))

pretty_float_bits(0.1)

'0 01111111011 1001100110011001100110011001100110011001100110011010'

In [49]:
pretty_float_bits(0.1 + 0.2)

'0 01111111101 0011001100110011001100110011001100110011001100110100'

In [50]:
pretty_float_bits(0.3)

'0 01111111101 0011001100110011001100110011001100110011001100110011'

With floats, the order matters:

In [75]:
100. - 100. + 0.1

0.1

In [76]:
0.1 + 100. - 100.

0.09999999999999432

The mantissa of 0.1 + 0.2 and 0.3 are not the same because of the propagation of an error bit at the end of the mantissa, since the computer has to register ints of 32 bits. Thus the error when doing 0.1 + 0.2

#### 3) Machine precision and cumulative errors

Consider the integral $$I_n = \int_{0}^1 \frac{x^n}{x + 10}dx$$

1. Without computing $I_n$, find its limit.
2. Compute $I_0$ and find a recurrence formula between $I_{n+1}$ and $I_n$
3. If we were to compute $I_n$ recursively, would that algorithm be stable numerically ?

___

1] We have :
$$I_n = \int_{0}^1 \frac{x^n}{x + 10}dx \leq \int_{0}^1 \frac{x^n}{10}dx = \frac{1}{10(n+1)} $$ 
$$\lim_{n \to \infty} I_n = 0$$

2] $$I_0 = \ln(11/10)$$

We compute the integral with a parameter $a > 1$
For the computations in this part, we assume a = 10, and  $n \geq 1$.
$$I_{n+1} = \int_{0}^1 \frac{x^{n+1}}{x + a}dx$$

We integrate by parts :

$$\int_{0}^1 \frac{x^{n+1}}{x + a}dx = [x^{n+1}\ln(x+a)]_0^1 - \int_{0}^1 (n+1)x^n\ln(x+a) dx$$

$$I_{n+1} = \ln(a+1) - (n+1)\int_{0}^1 x^n\ln(x+a) dx$$

We want to compute $\int_{0}^1 x^n\ln(x+a) dx$. We integrate by parts :

$$\int_{0}^1 x^n\ln(x+a) dx = [x^n(x+a)(\ln(x+a)-1)]_0^1 - \int_{0}^1 nx^{n-1}(x+a)(\ln(x+a)-1) dx$$

$$\int_{0}^1 x^n\ln(x+a) dx = (1+a)(\ln(1+a)-1) - n\int_{0}^1 x^{n-1}(x+a)(\ln(x+a)-1) dx$$

$$I_{n+1} = \ln(a+1) - (n+1)(1+a)(\ln(1+a)-1) + n(n+1)\int_{0}^1 x^{n-1}(x+a)(\ln(x+a)-1) dx$$

Since we have, $\forall n \geq 1$, $I_{n} = \ln(a+1) - n\int_{0}^1 x^{n-1}\ln(x+a) dx$

$$n(n+1)\int_{0}^1 x^{n-1}(x+a)(\ln(x+a)-1) dx = n(n+1)\int_{0}^1 x^n(\ln(x+a)-1) dx + an(n+1)\int_{0}^1 x^{n-1}(\ln(x+a)-1) dx $$
$$n(n+1)\int_{0}^1 x^{n-1}(x+a)(\ln(x+a)-1) dx = n(n+1)\int_{0}^1 x^n\ln(x+a) dx - n(n+1)\int_{0}^1 x^n dx + an(n+1)\int_{0}^1 x^{n-1}\ln(x+a) dx - an(n+1)\int_{0}^1 x^{n-1} dx $$

$$n(n+1)\int_{0}^1 x^{n-1}(x+a)(\ln(x+a)-1) dx = n(\ln(1+a)-I_{n+1}) - n + a(n+1)(\ln(1+a)-I_{n}) - a(n+1)$$

Thus :

$$I_{n+1} = \ln(a+1) - (n+1)(1+a)(\ln(1+a)-1) + n(\ln(a+1)-I_{n+1}) - n + a(n+1)(\ln(1+a)-I_{n}) - a(n+1)$$
$$I_{n+1}(n+1) = \ln(a+1) - (n+1)(1+a)(\ln(1+a)-1) + n\ln(1+a) - n + a(n+1)(\ln(1+a)-I_{n}) - a(n+1)$$
$$I_{n+1}(n+1) = \ln(a+1) - (n+1)(1+a)\ln(1+a) + (n+1)(1+a) + n\ln(a+1) - n + a(n+1)\ln(1+a)-a(n+1)I_{n} - a(n+1)$$

$$I_{n+1}(n+1) = 1 - a(n+1)I_{n} + \ln(1+a) - (n+1)(1+a)\ln(1+a) + n\ln(1+a) + a(n+1)\ln(1+a)$$

$$I_{n+1} = \frac{1}{n+1} - aI_{n}$$

___

Another way of obtaining this result is the following :

$$I_{n+1} = \int_{0}^1 \frac{x^{n+1}}{x + a}dx$$

$$I_{n+1} = \int_{0}^1 \frac{x^n(x+a) - ax^n}{x + a}dx$$

$$I_{n+1} = \int_{0}^1 x^n dx  - a \int_{0}^1 \frac{x^n}{x + a}dx$$

$$I_{n+1} = \frac{1}{n+1}  - a I_n$$

___

In [59]:
def integral(i0, n):
    ii = i0
    for jj in range(n):
        ii = 1/(jj+1) - 10*ii
    return ii

In [64]:
for k in range(20):
    print(f"I{k} = {integral(np.log(11/10), k)}")

I0 = 0.09531017980432493
I1 = 0.04689820195675065
I2 = 0.031017980432493486
I3 = 0.023153529008398455
I4 = 0.018464709916015454
I5 = 0.015352900839845474
I6 = 0.013137658268211921
I7 = 0.011480560175023635
I8 = 0.010194398249763648
I9 = 0.009167128613474629
I10 = 0.008328713865253717
I11 = 0.007621952256553738
I12 = 0.00711381076779595
I13 = 0.005784969245117427
I14 = 0.013578878977397152
I15 = -0.06912212310730485
I16 = 0.7537212310730486
I17 = -7.478388781318721
I18 = 74.83944336874276
I19 = -748.3418021084802


4. Replace 10 in the integral with a constant A > 1. Given a machine precision variable $\varepsilon$, how can we set the number of iterations $n$ based on a desired cumulative error E ?

___

We consider a machine precision variable $\varepsilon > 0$. That means that :

$$|I_1 - (\frac{1}{2} - AI_0)| = \varepsilon$$

We can write it as :

$$ \frac{1}{2} - AI_0 - \varepsilon \leq I_1 \leq \frac{1}{2} - AI_0 + \varepsilon $$

We apply the recurrence formula on this equality :

$$ \frac{1}{3} - A(\frac{1}{2} - AI_0 + \varepsilon) \leq I_2 \leq \frac{1}{3} - A(\frac{1}{2} - AI_0 - \varepsilon) $$

We add the machine precision variable for those calculations :

$$ \frac{1}{3} - A(\frac{1}{2} - AI_0 + \varepsilon) - \varepsilon \leq I_2 \leq \frac{1}{3} - A(\frac{1}{2} - AI_0 - \varepsilon) + \varepsilon $$

And we simplify to express $I_2$ with I_1 and the error.

$$ \frac{1}{3} - AI_1  - (A+1)\varepsilon \leq I_2 \leq \frac{1}{3} - AI_1 + (A+1)\varepsilon $$

We can then show recursively (on this model) that the cumulative error $E_n$ to compute $I_n$ can be expressed as follows :

$$ E_n = \varepsilon \sum_{k=0}^{n-1} A^k $$

We want to find $n$ such that $E_{n+1} \geq E \geq E_n$, which means that the number of iterations $n$ is based on the desired cumulative error E.

$$ E_n \leq E $$

$$ \varepsilon \sum_{k=0}^{n-1} A^k \leq E $$

We recognize a finite geometric sum (the formula works $\forall A \in \textbf{R}^*$ as long as the sum is finite) :

$$ \frac{1-A^n}{1-A} \leq \frac{E}{\varepsilon} $$

$$ \frac{A^n-1}{A-1} \leq \frac{E}{\varepsilon} $$

$$ A^n \leq \frac{(A-1)E}{\varepsilon} + 1 $$

We take the natural logarithm which is an increasing function on $\textbf{R}_+^*$ :

$$ n \leq \frac{\ln(\frac{(A-1)E}{\varepsilon} + 1)}{\ln(A)} $$

$$ n = \left\lfloor  \frac{\ln(\frac{(A-1)E}{\varepsilon} + 1)}{\ln(A)} \right\rfloor $$



In [ ]:
A = 10
E = 0.001
n = np.floor(np.log((A-1)*E/np.finfo(float).eps + 1)/np.log(A))
print(n)

13.0


If A = 10, and the cumulative error must not exceed 0.001, then the max number of iterations is 13 before the cumulative error exceed E.

In [65]:
print(np.finfo(float).eps) 

2.220446049250313e-16


**Independent questions:**

**Q5.** Write a piece of code that can find $\varepsilon$ numerically. 



In [80]:
eps = 1.0
while 1.0 + eps != 1.0 :
    eps /= 2 # We use power of two since the machine errors is stored using bits
eps *= 2 # We want the last value of epsilon before epsilon is considered null by machine approximation
print(eps)

2.220446049250313e-16


We get the same value as the function np.finfo(float).eps

**Q6.** Given what you know now, how should you test if two numbers or arrays are equal ? 

Two numbers a and b are equal numerically if and only if $|a-b| \leq \varepsilon$  

#### 4) Logsum-exp trick
Consider a classification model with 4 classes. We are modeling the probablity of a sample being in class $k$ with: $$p_k = \frac{ exp(w_k)}{\sum_{i=1}^{4} exp(w_i)}$$

where $w$ are the weights of a neural net.
1. Why does this model make sense ? 
2. Given the example $w = [-20, -1, 0, 1000]$, it is obvious which class this sample should correspond to. Compute the prediction probabilities using the function below for this particular example. Explain.

___

1. This model makes sense because p is a probability, as $\sum_{k=1}^{4} p_k = \sum_{k=1}^{4} \frac{ exp(w_k)}{\sum_{i=1}^{4} exp(w_i)} = 1$ 

In [81]:
def predict(w):
    p = np.exp(w)
    p /= p.sum()
    return p

w = np.array([10, -1, 40, 2, 1000])
predict(w)

C:\Users\Arnaud\AppData\Local\Temp\ipykernel_24284\437721929.py:2: RuntimeWarning: overflow encountered in exp
  p = np.exp(w)
C:\Users\Arnaud\AppData\Local\Temp\ipykernel_24284\437721929.py:3: RuntimeWarning: invalid value encountered in divide
  p /= p.sum()


array([ 0.,  0.,  0.,  0., nan])

We have an overflow error, as $e^{1000}$ cannot be stored in a 32 bits float.

3. Even if we assume that $exp(w_k)$ do not overflow, computing the normalizing sum can cause problems if the number of labels is too large. After showing the following statement, propose a method to modify `predict` in order to avoid overflow errors:
$$ \forall c \in \mathbb{R} \qquad log\left(\sum_{k=1}^K exp(w_k + c)\right) =  c + log\left(\sum_{k=1}^K exp(w_k)\right) $$ 
___

What we can do it use the above trick with $ c = - max_{k \in \textbf{N}} (w_k)$ to normalize all coefficients and remove the overflow possibility, since all the values will be stricly inferior to 1 after rescaling, and the final model will remain the same after the shift.

If we denote $ w = \max _{k \in \textbf{N}} w_k $ :

$$p_k = \frac{ \exp(w_k)}{\sum_{i=1}^{4} \exp(w_i)}$$

$$p_k = \frac{ \exp(w_k - w)}{\sum_{i=1}^{4} \exp(w_i - w)}$$

In [82]:
def predict_stable(w):
    w_scaled = w - np.max(w)
    return (np.exp(w_scaled)/(np.sum(np.exp(w_scaled))))

In [83]:
predict_stable(w)

array([0., 0., 0., 0., 1.])

4. Generate random weight vectors with `np.random.randn(K)` and test that both functions return the same probabilities. Test it with the scipy `logsumexp`: 

In [142]:
K = 10
vect = np.random.randn(K)
print(f"Random vector : {vect}")
print(f"Predict function result : {predict(vect)}")
print(f"Predict stable function result : {predict_stable(vect)}")

Random vector : [-0.80173116 -2.69293807  2.40773644 -1.96127722 -1.51013622  0.20341492
 -0.56683723  1.81581674 -0.66458307 -0.88107965]
Predict function result : [0.02150874 0.00324545 0.53268328 0.00674575 0.01059153 0.05876847
 0.02720371 0.29471457 0.02467048 0.01986801]
Predict stable function result : [0.02150874 0.00324545 0.53268328 0.00674575 0.01059153 0.05876847
 0.02720371 0.29471457 0.02467048 0.01986801]


The results are the same.